In [64]:
import json
from pprint import pprint
import glob
import os
import re
import cv2
import pandas as pd
import numpy as np
import sys

In [65]:
class Points(): 
    def __init__(self):
        self.frameNuber = -1
        self.xLeft = []
        self.xRight = []
        self.yLeft = []
        self.yRight = []
        self.cLeft = []
        self.cRight = []
        self.xBody = []
        self.yBody = []
        self.cBody = []
        self.minDist = sys.float_info.max
    

In [66]:
class SortedCutPoint:
    def __init__(self):
        self.idxDistTransA = -1
        self.idxDistTransB = -1
        self.totalDifIdx = -1
        self.frameIdx = -1

In [67]:
def LoadTransition():
    trans = []
#     point_ins = Points()
#     point_ins.xLeft = []
    files = glob.glob("/home/yulia/openpose/transition3/*.json")
    ctrTrans = 0
    for f in files:
        fName = os.path.basename(f)
        number = re.split("(\d+)", fName)
        number = int(number[3])
        trans.append(Points())
        with open(f) as df:
            data = json.load(df)
            frameNumber = number
            trans[ctrTrans].frameNumber = number
#             print(number)
            #each hand has 21 points. x,y,c --> 0 to 62
            for i in range(0,21):
                trans[ctrTrans].xLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3])
                trans[ctrTrans].yLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+1])
                trans[ctrTrans].cLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+2])
                trans[ctrTrans].xRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3])
                trans[ctrTrans].yRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+1])
                trans[ctrTrans].cRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+2])
            for i in range(0, 25):
                trans[ctrTrans].xBody.append(data['people'][0]['pose_keypoints_2d'][i*3])
                trans[ctrTrans].yBody.append(data['people'][0]['pose_keypoints_2d'][i*3+1])
                trans[ctrTrans].cBody.append(data['people'][0]['pose_keypoints_2d'][i*3+2])
        ctrTrans+=1
    return trans

In [68]:
trans = LoadTransition()

In [69]:
wordsFrame = []
wordsFrame.append(Points())
ctrWordsFrame = 0
with open("/home/yulia/openpose/baby/baby_000000000032_keypoints.json") as df:
    data = json.load(df)
    wordsFrame[ctrWordsFrame].frameNumber = ctrWordsFrame
    #each hand has 21 points. x,y,c --> 0 to 62
    for i in range(0,21):
        wordsFrame[ctrWordsFrame].xLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+2])
        wordsFrame[ctrWordsFrame].xRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+2])
    for i in range(0, 25):
        wordsFrame[ctrWordsFrame].xBody.append(data['people'][0]['pose_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yBody.append(data['people'][0]['pose_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cBody.append(data['people'][0]['pose_keypoints_2d'][i*3+2])

ctrWordsFrame+=1
wordsFrame.append(Points())
with open("/home/yulia/openpose/awake/awake_000000000000_keypoints.json") as df:
    data = json.load(df)
    wordsFrame[ctrWordsFrame].frameNumber = ctrWordsFrame

    #each hand has 21 points. x,y,c --> 0 to 62
    for i in range(0,21):
        wordsFrame[ctrWordsFrame].xLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cLeft.append(data['people'][0]['hand_left_keypoints_2d'][i*3+2])
        wordsFrame[ctrWordsFrame].xRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cRight.append(data['people'][0]['hand_right_keypoints_2d'][i*3+2])
    for i in range(0, 25):
        wordsFrame[ctrWordsFrame].xBody.append(data['people'][0]['pose_keypoints_2d'][i*3])
        wordsFrame[ctrWordsFrame].yBody.append(data['people'][0]['pose_keypoints_2d'][i*3+1])
        wordsFrame[ctrWordsFrame].cBody.append(data['people'][0]['pose_keypoints_2d'][i*3+2])

In [70]:
class Distance(object):
    def __init__(self):
        self.fiX = []
        self.fiY = []
        self.hjX = []
        self.hjY = []
        self.fiSubHj = []
        self.distance = 0
        self.From = -1
        self.To = -1
        self.wordFrom = -1
        self.wordTo = -1

In [188]:
class BallEquation(object):
    def __init__(self):
        self.point_pairs = ['0,1', '0,5', '0,9', '0,13', '0,17']
        self.radius_list = []
        self.radius_origin_scene = []

In [72]:
tempTrans = trans.copy()

#make the range for the Y position of the palm with index point = 0
if wordsFrame[0].yLeft[0] > wordsFrame[1].yLeft[0]:
    yLeftMin = wordsFrame[1].yLeft[0]
    yLeftMax = wordsFrame[0].yLeft[0]
else:
    yLeftMin = wordsFrame[0].yLeft[0]
    yLeftMax = wordsFrame[1].yLeft[0]
if wordsFrame[0].yRight[0] > wordsFrame[1].yRight[0]:
    yRightMin = wordsFrame[1].yRight[0]
    yRightMax = wordsFrame[0].yRight[0]
else:
    yRightMin = wordsFrame[0].yRight[0]
    yRightMax = wordsFrame[1].yRight[0]

filteredTransByY = list(filter(lambda x : x.yLeft[0] <= yLeftMax and x.yLeft[0] >= yLeftMin and x.yRight[0] <= yRightMax and x.yRight[0] >= yRightMin, tempTrans))

In [73]:
filteredTransByY[0].frameNumber

177

In [193]:
ball = []
for w in range(0,2):
    ball.append(BallEquation())
    for i in range(0,2): #2 hands
        if i == 0:
            handX = wordsFrame[w].xLeft
            handY = wordsFrame[w].yLeft
        else:
            handX = wordsFrame[w].xRight
            handY = wordsFrame[w].yRight

        for j in range(0,5): #5 fingers
            center_point_x = handX[(int)(ball[w].point_pairs[j].split(',')[0])]
            center_point_y = handY[(int)(ball[w].point_pairs[j].split(',')[0])]
            related_point_x = handX[(int)(ball[w].point_pairs[j].split(',')[1])]
            related_point_y = handY[(int)(ball[w].point_pairs[j].split(',')[1])]

            for k in range(0,4): #4 ball equations for each finger
                rel_x = np.round(related_point_x - center_point_x)
                rel_y = np.round(related_point_y - center_point_y)
                radius = np.power(rel_x,2)+np.power(rel_y,2)
                ball[w].radius_list.append(radius)
#                 rel_z = np.sqrt(radius-np.power(rel_x,2)-np.power(rel_y,2))
                if(k < 3):
                    center_point_x = related_point_x
                    center_point_y = related_point_y
                    related_point_x = handX[(int)(ball[w].point_pairs[j].split(',')[1])+k+1]
                    related_point_y = handY[(int)(ball[w].point_pairs[j].split(',')[1])+k+1]
            

In [196]:
balls=[]
for i in range(0,2):
    balls.append(ball[i].radius_list)
hand_ball_equation = []
hand_ball_equation = BallEquation()
hand_ball_equation.radius_list.append(np.amax(balls,axis=0))
hand_ball_equation.radius_origin_scene.append(np.argmax(balls,axis=0))

[array([1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0])]

In [234]:
(int)(hand_ball_equation.point_pairs[17%20//4].split(',')[1])+(17%20%4)

18

In [247]:
13%20%4

1

In [250]:
for i in range(0,40):
    if hand_ball_equation.radius_origin_scene[0][i] == 0:
        scene = 1
    else:
        scene = 0
    if i < 20:
        handX = wordsFrame[scene].xLeft
        handY = wordsFrame[scene].yLeft
    else:
        handX = wordsFrame[scene].xRight
        handY = wordsFrame[scene].yRight

    if i%20%4 == 0:
        center_point_x = handX[(int)(hand_ball_equation.point_pairs[i%20//4].split(',')[0])]
        center_point_y = handY[(int)(hand_ball_equation.point_pairs[i%20//4].split(',')[0])]
        related_point_x = handX[(int)(hand_ball_equation.point_pairs[i%20//4].split(',')[1])]
        related_point_y = handY[(int)(hand_ball_equation.point_pairs[i%20//4].split(',')[1])]
    else:
        center_point_x = handX[(int)(hand_ball_equation.point_pairs[i%20//4].split(',')[1])+(i%20%4)-1]
        center_point_y = handY[(int)(hand_ball_equation.point_pairs[i%20//4].split(',')[1])+(i%20%4)-1]
        related_point_x = handX[(int)(hand_ball_equation.point_pairs[i%20//4].split(',')[1])+(i%20%4)]
        related_point_y = handY[(int)(hand_ball_equation.point_pairs[i%20//4].split(',')[1])+(i%20%4)]
#         print((int)(hand_ball_equation.point_pairs[i%20//4].split(',')[1])+(i%20%4)-1)
#         print((int)(hand_ball_equation.point_pairs[i%20//4].split(',')[1])+(i%20%4))
        
    rel_x = np.round(related_point_x - center_point_x)
    rel_y = np.round(related_point_y - center_point_y)
    radius = hand_ball_equation.radius_list[0][i]
    rel_z = np.sqrt(radius-np.power(rel_x,2)-np.power(rel_y,2))
    print(rel_z)
#     print(radius)
#     print(rel_x)
#     print(np.power(rel_x,2))
#     print(rel_y)
#     print(np.power(rel_y,2))
#     print(radius-np.power(rel_x,2)-np.power(rel_y,2))
#     print()

15.556349186104045
13.341664064126334
17.72004514666935
3.4641016151377544
26.49528259898354
25.768197453450252
10.392304845413264
13.96424004376894
17.05872210923198
25.298221281347036
7.810249675906654
14.798648586948742
31.0
26.94438717061496
17.578395831246947
9.38083151964686
34.10278580995987
22.47220505424423
5.744562646538029
7.211102550927978
17.320508075688775
8.246211251235321
31.827660925679098
51.633322573702344
32.12475680841802
31.622776601683793
5.385164807134504
16.64331697709324
60.654760736483
17.175564037317667
10.770329614269007
12.96148139681572
67.6904720030818
12.041594578792296
22.80350850198276
40.93897898091744
75.67694497005016
18.24828759089466
10.63014581273465
51.478150704935004


In [ ]:
#order the Y value of the palm keypoint number 0 so that the direction of the palm translation is ordered correctly
#find the most dominant moving hand, since there are 2 hands need to be concerned. if the movement is bigger, consider it as the key to order the list
if yRightMax - yRightMin > yLeftMax - yLeftMin:
    

In [ ]:
# transFrame = []
# dist = []
# transFrame.append(wordsFrame[0])
# transFrame.append(wordsFrame[1])
# tempTrans = trans.copy()
# tempTrans.append(transFrame[0])
# tempTrans.append(transFrame[1])
# idx = 0`

# for i in range(0,len(tempTrans)):
#     for j in range(i+1, len(tempTrans)):
#         dist.append(Distance())
#         for k in range(0, 21):
#             xL1 = tempTrans[i].xLeft[k]
#             yL1 = tempTrans[i].yLeft[k]
#             xR1 = tempTrans[i].xRight[k]
#             yR1 = tempTrans[i].yRight[k]
#             xL2 = tempTrans[j].xLeft[k]
#             yL2 = tempTrans[j].yLeft[k]
#             xR2 = tempTrans[j].xRight[k]
#             yR2 = tempTrans[j].yRight[k]

#             dist[idx].fiX.append(xL1)
#             dist[idx].fiX.append(xR1)

#             dist[idx].hjX.append(xL2)
#             dist[idx].hjX.append(xR2)

#             dist[idx].fiY.append(yL1)
#             dist[idx].fiY.append(yR1)

#             dist[idx].hjY.append(yL2)
#             dist[idx].hjY.append(yR2)

#             dist1 = np.absolute(xL1-xL2)+np.absolute(yL1 - yL2)
#             dist[idx].fiSubHj.append(dist1)

#             dist2 = np.absolute(xR1 - xR2)+np.absolute(yR1 - yR2)
#             dist[idx].fiSubHj.append(dist2)

#         #get value at the minimum index of the substraction result ||fi-hj||
#         minXI = dist[idx].fiX[dist[idx].fiSubHj.index(np.min(dist[idx].fiSubHj))]
#         minYI = dist[idx].fiY[dist[idx].fiSubHj.index(np.min(dist[idx].fiSubHj))]
#         minXJ = dist[idx].hjX[dist[idx].fiSubHj.index(np.min(dist[idx].fiSubHj))]
#         minYJ = dist[idx].hjY[dist[idx].fiSubHj.index(np.min(dist[idx].fiSubHj))]

#         fiMinHj = 0
#         minFiHj = 0
#         for k in range(0, 42):
#             fiMinHj += np.absolute(dist[idx].fiX[k] - minXJ) + np.absolute(dist[idx].fiY[k] - minYJ)
#             minFiHj += np.absolute(minXI - dist[idx].hjX[k]) + np.absolute(minYI - dist[idx].hjY[k])

#         dist[idx].distance = fiMinHj / len(dist[idx].fiSubHj) + minFiHj / len(dist[idx].fiSubHj)
#         dist[idx].From = tempTrans[i].frameNumber
#         dist[idx].To = tempTrans[j].frameNumber
        
#         if tempTrans[i].minDist > dist[idx].distance:
#             tempTrans[i].minDist = dist[idx].distance
#         if tempTrans[j].minDist > dist[idx].distance:
#             tempTrans[j].minDist = dist[idx].distance
        
#         if i >len(tempTrans)-3:
#             dist[idx].wordFrom = tempTrans[i].frameNumber
#         if j > len(tempTrans)-3:
#             dist[idx].wordTo = tempTrans[j].frameNumber
#         idx+=1

In [46]:
def readVideo(vid):
    cap = vid
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
    
    fc = 0
    ret = True

    while (fc < frameCount  and ret):
        ret, buf[fc] = cap.read()
        fc += 1

    cap.release()
    return buf

In [47]:
#read transition video and save to array of images
vidTrans = cv2.VideoCapture('/home/yulia/Documents/transition3.mp4')
vidBaby = cv2.VideoCapture('/home/yulia/Documents/ASL/baby.mp4')
vidAwake = cv2.VideoCapture('/home/yulia/Documents/ASL/awake.mp4')
transImg = readVideo(vidTrans)

In [48]:
wordsVid = []
wordsVid.append(readVideo(vidBaby))
wordsVid.append(readVideo(vidAwake))

In [51]:
writtenTrans = []
for i in range(0, len(filteredTransByY)):
    writtenTrans.append(transImg[filteredTransByY[i].frameNumber])

In [222]:
writtenTrans = []
# transFrame.pop(0)
# transFrame.pop(len(transFrame)-1)
for i in range(1,len(arr_path)-1):
    writtenTrans.append(transImg[arr_path[i].frameNumber])
    print(arr_path[i].frameNumber)
# for i in range(0,sortedCutPoint[0].idxDistTransA):
#     writtenTrans.append(transImg[dist[0][i].idxTrans])
# for i in range(sortedCutPoint[0].idxDistTransB-1, -1, -1):
#     writtenTrans.append(transImg[dist[1][i].idxTrans])

29
113
30
115
17
77
104
42


In [52]:
totalWrittenFrames = []
totalWrittenFrames.append(wordsVid[0])
totalWrittenFrames.append(writtenTrans)
totalWrittenFrames.append(wordsVid[1])

In [54]:
height,width,channels = wordsVid[0][0].shape
out = cv2.VideoWriter('/home/yulia/palm/filteredY.mp4',cv2.VideoWriter_fourcc('F','M','P', '4'), 10, (width,height))
# out = cv2.VideoWriter('/home/yulia/outpyMax2.mp4',cv2.VideoWriter_fourcc('F','M','P', '4'), 10, (width,height))
for i in range(0, len(totalWrittenFrames)):
    for frame in range(0, len(totalWrittenFrames[i])):
        out.write(totalWrittenFrames[i][frame])
        print(frame)
out.release()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
0
1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
len(totalWrittenFrames[1])